# Implementation of face recognition using neural net 

In [1]:
%matplotlib inline

In [2]:
import cv2
import numpy as np
import os
from skimage import io
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

/home/jjyeh/work/notebooks/PyData/_venv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
os.environ["KERAS_BACKEND"]="theano"
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

Using Theano backend.


#### Listing the path of all the images

In [4]:
DatasetPath = []
for i in os.listdir("yalefaces"):
    DatasetPath.append(os.path.join("yalefaces", i))

#### Reading each image and assigning respective labels

In [5]:
imageData = []
imageLabels = []

In [6]:
for i in DatasetPath:
    imgRead = io.imread(i,as_grey=True)
    imageData.append(imgRead)
    
    labelRead = int(os.path.split(i)[1].split(".")[0].replace("subject", "")) - 1
    imageLabels.append(labelRead)

#### Preprocessing: Face Detection using OpenCV and cropping the image to a size of 150 * 150

In [7]:
faceDetectClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [8]:
imageDataFin = []
for i in imageData:
    facePoints = faceDetectClassifier.detectMultiScale(i)
    x,y = facePoints[0][:2]
    cropped = i[y: y + 150, x: x + 150]
    imageDataFin.append(cropped)

In [9]:
c = np.array(imageDataFin)

In [10]:
c.shape

(165, 150, 150)

#### Splitting Dataset into train and test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(np.array(imageDataFin),np.array(imageLabels), train_size=0.9, random_state = 20)

In [12]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [13]:
X_train.shape

(148, 150, 150)

In [14]:
X_test.shape

(17, 150, 150)

In [15]:
nb_classes = 15
y_train = np.array(y_train) 
y_test = np.array(y_test) 

In [16]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

#### Converting each 2d image into 1D vector

In [17]:
X_train = X_train.reshape(148, 150*150)
X_test = X_test.reshape(17, 150*150)

In [18]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [19]:
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

('Training matrix shape', (148, 22500))
('Testing matrix shape', (17, 22500))


#### Defining the model

In [20]:
model = Sequential()
model.add(Dense(512,input_shape=(X_train.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               11520512  
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                7695      
__________

In [22]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

#### Training

In [23]:
model.fit(X_train, Y_train, batch_size=64, nb_epoch=50, verbose=1, validation_data=(X_test, Y_test))

/home/jjyeh/work/notebooks/PyData/_venv/local/lib/python2.7/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 148 samples, validate on 17 samples
Epoch 1/50
148/148 [==============================] - 0s - loss: 8.2607 - acc: 0.0743 - val_loss: 12.9135 - val_acc: 0.0588
Epoch 2/50
148/148 [==============================] - 0s - loss: 12.3105 - acc: 0.0878 - val_loss: 11.1807 - val_acc: 0.1765
Epoch 3/50
148/148 [==============================] - 0s - loss: 11.7238 - acc: 0.1351 - val_loss: 12.6105 - val_acc: 0.1176
Epoch 4/50
148/148 [==============================] - 0s - loss: 11.3278 - acc: 0.1689 - val_loss: 13.1973 - val_acc: 0.1176
Epoch 5/50
148/148 [==============================] - 1s - loss: 10.8285 - acc: 0.1959 - val_loss: 10.5864 - val_acc: 0.2353
Epoch 6/50
148/148 [==============================] - 0s - loss: 11.9846 - acc: 0.1622 - val_loss: 10.7130 - val_acc: 0.2941
Epoch 7/50
148/148 [==============================] - 0s - loss: 11.6133 - acc: 0.1622 - val_loss: 10.9083 - val_acc: 0.2353
Epoch 8/50
148/148 [==============================] - 0s - loss: 11.2276 - acc: 0

#### Evaluating the performance

In [24]:
loss, accuracy = model.evaluate(X_test,Y_test, verbose=0)

In [25]:
loss

0.54689353704452515

In [26]:
accuracy

0.76470589637756348

In [27]:
predicted_classes = model.predict_classes(X_test)

17/17 [==============================] - 0s


In [28]:
correct_classified_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_classified_indices = np.nonzero(predicted_classes != y_test)[0]

In [29]:
correct_classified_indices

array([ 0,  1,  3,  4,  5,  7,  8,  9, 10, 11, 12, 14, 15])

In [30]:
incorrect_classified_indices

array([ 2,  6, 13, 16])